In [193]:
import string
import math
from collections import OrderedDict
import numpy as np
from operator import itemgetter

# Funções

In [493]:
def multVet(d,q):
    soma = 0
    for i in range(0,len(q)):
        soma += d[i]*q[i]
    return soma

def norma(v):
    soma = 0
    for i in range(0,len(v)):
        soma += v[i]**2
    return math.sqrt(soma)

def sumStr(s):
    return sum(map(int, s))

def getRelevantes(qn):
    relevantes = perguntas[qn][perguntas[qn].index('RD')+2:].split()
    docsRlv = []
    for i in range(0,len(relevantes)-1,2):
        docsRlv.append([relevantes[i],sumStr(relevantes[i+1])])
    return docsRlv

def getAP(tops,relevantes):
    acm = 0
    lidos = 0
    encontrados = 0
    for i in range(0,len(tops)):
        lidos += 1
        if str(tops[i]+1) in relevantes:
            encontrados += 1
            acm += encontrados/lidos
    return acm/len(relevantes)

def getTopN(n, sim):
    return np.argsort(sim)[-n:]

def getPerguntas():
    queries = []
    arqPerguntas = open('cfc/cfquery','r')
    perguntas = arqPerguntas.read()
    perguntas = perguntas.split('QN')

    for i in range(1, len(perguntas)):
        queries.append(perguntas[i][perguntas[i].index('QU')+3:perguntas[i].index('NR')].strip(string.punctuation).lower().replace("?",'').replace(',','').split())
    return queries

def getVetorDocs(docs, listaInv):
    listaInv = {}
    docs = []
    for i in range(0,6):
        arq = open('cfc/' + cfcs[i])
        documentos = arq.read()
        documentos = documentos.split('\n\n')
        
        for documento in documentos:
            voc = {}
            if "" in documento:
                break
            if len(documento) < 5:
                #print('erro')
                continue
            else:
                try:
                    autor = documento[documento.index('\nAU')+3:documento.index('\nTI')].split()
                except:
                    autor = []

                try:
                    title = documento[documento.index('\nTI')+3:documento.index('\nSO')].split()
                except:
                    title = []

    #             try:
    #                 fonte = documento[documento.index('\nSO')+3:documento.index('\nMJ')].split()
    #             except:
    #                 fonte = []

                try:
                    mj = documento[documento.index('\nMJ')+3:documento.index('\nMN')].split()
                except:
                    mj = []

                try:
                    mn = documento[documento.index('MN')+3:documento.index('\nAB')].split()
                except:
                    mn = documento[documento.index('MN')+3:documento.index('\nEX')].split() 

                try:
                    try:
                        ab = documento[documento.index('\nAB')+3:documento.index('\nRF')].split()
                    except:
                        ab = documento[documento.index('\nAB')+3:documento.index('\nCT')].split()                  
                except:
                    try:
                        ab = documento[documento.index('\nEX')+3:documento.index('\nRF')].split()
                    except:
                        try:
                            ab = documento[documento.index('\nEX')+3:].split()
                        except:
                            ab = documento[documento.index('\nAB')+3:].split()


                for w in title + mj + mn + ab + autor:
                    #print(w.strip(string.punctuation).lower())
                    w = w.strip(string.punctuation).lower()
                    if not w.isdigit():
                        if w in voc:
                            voc[w] += 1
                        else:
                            voc[w] = 1
                for w in voc:
                    if w in listaInv:
                        listaInv[w].append([len(docs), voc[w]])
                    else:
                        listaInv[w] = [[len(docs),voc[w]]]
                docs.append(voc)
    return docs,listaInv

def vetConsultas(queries):
    vetQueries = []
    for pergunta in queries:
        palavrasLidas = []
        v = []
        for w in pergunta:
            if w not in palavrasLidas:
                try:
                    v.append(idfs[w] * pergunta.count(w))
                except:
                    v.append(0)
            palavrasLidas.append(w)
        vetQueries.append(v)
        #pergunta = list(OrderedDict.fromkeys(pergunta))
    return vetQueries

def geraVetDoc(pergunta):
    vetDocs = []
    pergunta = list(OrderedDict.fromkeys(pergunta))
    
    for doc in docs:
        v = []
        for w in pergunta:
            if w in doc:
                v.append(doc[w] * idfs[w])
            else:
                v.append(0)
        vetDocs.append(v)
    return vetDocs

def getSimilaridades(docs, vetDocs, vetQueries, qn):
    sim = []
    for i in range(0,len(docs)):
        try:
            sim.append(multVet(vetDocs[i],vetQueries[qn])/(norma(vetDocs[i])*norma(vetQueries[qn])))
        except:
            sim.append(0)
    return sim

# variáveis globais

In [450]:
cfcs = ['cf74','cf75','cf76','cf77','cf78','cf79']
listaInv = {} #dicionario que contém a lista invertida de todos os termos
docs = [] #vetor de documentos. Cada doc com seu dicionário que contem os termos(chave do dic) e suas frequencias
idfs = {} #dicionario que tem os termos como chave, e seu idf como valor
queries = [] #vetor com as perguntas
vetQueries = [] #vetores das consultas
vetDocs = [] #vetores dos documentos dado uma pergunta
sim = [] #vetor com a similiridade dos documentos com uma consulta

# adiciona as palavras no dicionário: listaInv
# e adiciona lista invertida dos documentos em: docs

In [451]:
docs, listaInv = getVetorDocs(docs,listaInv)

# gera o arquivo invertido

In [452]:
arquivoInv = open('arqInv.dat','w')
for i in range(0,len(docs)):
    linha = str(i)
    for w in docs[i]:
        linha += ';' + w + ' ' + str(docs[i][w])
    arquivoInv.writelines(linha + '\n')
arquivoInv.close()

# calcula o idf de todas as palavras

In [453]:
for w in listaInv:
    idfs[w] = math.log(len(docs)/len(listaInv[w]))

# list com todas as perguntas

In [454]:
queries = getPerguntas()

# list com vetores de cada consulta

In [530]:
vetQueries = vetConsultas(queries)

# list com vetores de documentos

In [532]:
vetDocs = geraVetDoc(queries[99])

# consultas

In [523]:
#CALCULANDO APs
acmAps = 0
for i in range(0,100):
    vetDocs = geraVetDoc(queries[i])
    sim = getSimilaridades(docs,vetDocs,vetQueries, i)
    relevantes = getRelevantes(i+1)
    vet = [item[0] for item in relevantes]
    top_n_idx = getTopN(len(vet),sim)
    top_n_idx = list(reversed(top_n_idx))
    acmAps += getAP(top_n_idx,vet)
    
    print('Consulta: ' + str(i+1))
    print('AP: ' + str(getAP(top_n_idx,vet)))
    top_n_idx = list(np.asarray(top_n_idx) + 1)
    print('relevantes:',end=' ')
    print(vet)
    print('tops:',end=' ')
    print(top_n_idx) 
    print()
    
print(acmAps) #MAP

Consulta: 1
AP: 0.25022566571385585
relevantes: ['139', '151', '166', '311', '370', '392', '439', '440', '441', '454', '461', '502', '503', '505', '520', '522', '526', '527', '533', '593', '619', '737', '742', '789', '827', '835', '861', '875', '891', '921', '922', '1175', '1185', '1222']
tops: [505, 533, 439, 943, 392, 311, 437, 875, 950, 52, 568, 593, 767, 490, 441, 106, 827, 461, 511, 497, 1218, 134, 139, 1148, 56, 499, 742, 911, 302, 465, 931, 1031, 957, 377]

Consulta: 2
AP: 0.03571428571428571
relevantes: ['169', '434', '454', '498', '499', '592', '875']
tops: [1197, 1125, 95, 592, 980, 137, 41]

Consulta: 3
AP: 0.06285486505405614
relevantes: ['23', '40', '139', '190', '221', '246', '309', '311', '325', '345', '347', '356', '370', '374', '375', '439', '440', '454', '515', '520', '524', '526', '527', '533', '535', '560', '561', '571', '584', '604', '623', '633', '733', '742', '854', '856', '950', '967', '1144', '1161', '1172', '1175', '1196']
tops: [633, 201, 430, 356, 1197, 515,

Consulta: 28
AP: 0.06585383561400311
relevantes: ['134', '135', '136', '215', '274', '300', '322', '331', '333', '343', '351', '352', '370', '392', '398', '465', '467', '477', '510', '550', '559', '586', '611', '665', '722', '724', '729', '735', '759', '763', '822', '832', '837', '875', '882', '913', '937', '942', '944', '969', '990', '1000', '1033', '1040', '1083', '1093', '1187', '1192', '1222', '1226', '1234']
tops: [1166, 807, 477, 610, 398, 1031, 1083, 352, 351, 56, 1148, 931, 943, 490, 767, 1111, 134, 298, 106, 1218, 911, 975, 735, 377, 926, 505, 465, 982, 427, 968, 898, 297, 13, 1019, 474, 402, 473, 422, 760, 725, 727, 752, 1084, 209, 350, 1169, 342, 665, 594, 369, 1094]

Consulta: 29
AP: 0.01989121989121989
relevantes: ['15', '100', '215', '274', '300', '322', '333', '343', '350', '351', '352', '353', '370', '385', '392', '434', '442', '477', '510', '553', '687', '714', '722', '724', '729', '759', '763', '764', '788', '822', '832', '837', '875', '937', '942', '944', '1000', '10

Consulta: 45
AP: 0.2523809523809524
relevantes: ['278', '283', '286', '287', '288', '289', '290', '292', '293', '338', '538', '540', '715', '758', '803', '919', '924', '1050', '1159', '1168']
tops: [919, 538, 287, 208, 1157, 803, 924, 490, 1159, 440, 133, 1148, 767, 465, 106, 961, 975, 377, 1031, 911]

Consulta: 46
AP: 0.1984126984126984
relevantes: ['51', '53', '370', '410', '430', '500', '503', '516', '553', '568', '623', '702', '737', '854', '856', '857', '1196', '1212']
tops: [53, 51, 1212, 405, 1148, 171, 856, 767, 1218, 975, 134, 931, 1031, 56, 437, 377, 911, 943]

Consulta: 47
AP: 0.027744861189791394
relevantes: ['15', '57', '92', '118', '121', '125', '163', '165', '172', '183', '208', '209', '245', '259', '278', '279', '283', '286', '287', '295', '296', '301', '302', '317', '337', '361', '370', '406', '416', '436', '505', '508', '525', '585', '613', '684', '694', '695', '707', '781', '862', '868', '906', '928', '931', '971', '998', '1000', '1013', '1058', '1075', '1106', '1107

Consulta: 62
AP: 0.1183548057352321
relevantes: ['10', '17', '24', '43', '56', '60', '62', '63', '64', '69', '72', '88', '94', '110', '144', '148', '155', '160', '182', '192', '193', '194', '197', '200', '211', '224', '226', '237', '238', '244', '267', '297', '321', '322', '330', '331', '333', '340', '348', '349', '354', '358', '359', '370', '372', '382', '383', '427', '429', '434', '437', '438', '446', '458', '459', '460', '465', '472', '473', '479', '486', '505', '537', '539', '550', '551', '558', '559', '580', '586', '590', '607', '643', '663', '677', '698', '703', '704', '722', '724', '729', '731', '734', '735', '736', '761', '763', '764', '767', '768', '783', '787', '803', '804', '806', '819', '821', '822', '837', '842', '843', '844', '845', '870', '879', '880', '882', '885', '886', '888', '889', '904', '907', '912', '913', '914', '920', '937', '941', '944', '945', '964', '969', '990', '1000', '1022', '1023', '1032', '1033', '1038', '1039', '1040', '1042', '1071', '1073', '1080', 

Consulta: 79
AP: 0.015454750183828872
relevantes: ['65', '75', '123', '132', '135', '146', '148', '179', '184', '255', '315', '320', '330', '333', '359', '370', '373', '384', '427', '438', '452', '648', '650', '663', '826', '829', '838', '882', '912', '923', '944', '945', '1000', '1010', '1023', '1065', '1082', '1089', '1091', '1112', '1179', '1181', '1234']
tops: [1101, 1066, 758, 815, 791, 396, 186, 238, 945, 735, 767, 1071, 1218, 337, 159, 734, 123, 30, 465, 458, 424, 460, 1091, 1148, 931, 912, 134, 943, 490, 1031, 56, 298, 1234, 106, 911, 975, 377, 926, 1111, 1198, 505, 616, 544]

Consulta: 80
AP: 0.22813894332436535
relevantes: ['61', '135', '152', '159', '238', '330', '333', '427', '438', '451', '452', '453', '458', '459', '460', '612', '663', '696', '722', '753', '800', '884', '912', '945', '1010', '1042', '1065', '1071', '1078', '1091', '1179', '1180', '1227', '1234']
tops: [945, 753, 1111, 696, 1089, 61, 943, 159, 460, 1042, 1078, 767, 1101, 135, 86, 782, 684, 1218, 546, 800, 

Consulta: 95
AP: 0.3055555555555556
relevantes: ['12', '30', '43', '192', '257', '322', '603', '606', '1097']
tops: [257, 30, 883, 606, 615, 348, 490, 133, 134]

Consulta: 96
AP: 0.22916666666666666
relevantes: ['9', '40', '43', '75', '454', '455', '520', '526', '527', '673', '883', '1087']
tops: [673, 455, 1088, 454, 29, 1084, 370, 473, 36, 782, 8, 866]

Consulta: 97
AP: 0.125
relevantes: ['52', '68', '135', '140', '190', '392', '416', '538', '539', '751', '757']
tops: [924, 135, 958, 751, 418, 44, 435, 416, 426, 1225, 287]

Consulta: 98
AP: 0.09307359307359307
relevantes: ['145', '180', '270', '272', '362', '467', '595', '667', '728', '782', '787', '985', '1018', '1019', '1118']
tops: [180, 360, 498, 505, 1143, 562, 1031, 106, 1079, 1156, 1118, 1157, 1184, 1018, 516]

Consulta: 99
AP: 0.8
relevantes: ['268', '324', '449', '992', '1191']
tops: [324, 268, 449, 992, 484]

Consulta: 100
AP: 0.2727272727272727
relevantes: ['59', '183', '370', '579', '803', '833', '1000', '1017', '1033', '